<h2>Importing Standard Libraries</h2>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h2>Loading Data</h2>

In [371]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [109]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,User Count
0,1000001,P00069042,0,15,10,A,2,0,3,NaN,NaN,8370,35
1,1000001,P00248942,0,15,10,A,2,0,1,6.0,14.0,15200,35
2,1000001,P00087842,0,15,10,A,2,0,12,NaN,NaN,1422,35
3,1000001,P00085442,0,15,10,A,2,0,12,14.0,NaN,1057,35
4,1000002,P00285442,1,60,16,C,4,0,8,NaN,NaN,7969,77


<h2> Feature Engineering </h2>

In [372]:
userCount = df.User_ID.value_counts().to_dict()
userCount_test = df_test.User_ID.value_counts().to_dict()

In [373]:
def UserCounts(x):
    return userCount[x]

def UserCounts_test(x):
    return userCount_test[x]

In [374]:
df['User Count'] = df['User_ID'].apply(UserCounts)
df_test['User Count'] = df_test['User_ID'].apply(UserCounts_test)

In [102]:
def Gender(x):
    if x == 'F':
        return 0
    else:
        return 1


In [375]:
df['Gender'] = df['Gender'].apply(Gender)
df_test['Gender'] = df_test['Gender'].apply(Gender)

In [104]:
def ageConverter(x):
    if x == '0-17':
        x=15
    elif x == "18-25":
        x=21
    elif x == "26-35":
        x=30
    elif x == "36-45":
        x=40
    elif x == "46-50":
        x=48
    elif x == "51-55":
        x=53
    elif x == "55+":
        x=60
    return x

In [395]:
df['Age']=df['Age'].apply(ageConverter)
df_test['Age']=df_test['Age'].apply(ageConverter)

In [106]:
def cityConverter(x):
    if x=='0':
        x=0
    elif x=='1':
        x=1
    elif x=='2':
        x=2
    elif x=='3':
        x=3
    if x=='4+':
        x=4
    return x

In [377]:
df["Stay_In_Current_City_Years"]=df["Stay_In_Current_City_Years"].apply(cityConverter)
df_test["Stay_In_Current_City_Years"]=df_test["Stay_In_Current_City_Years"].apply(cityConverter)

In [379]:
ProductAverages=df.groupby('Product_ID').mean()['Purchase'].to_dict()

In [381]:
def ProductMean(x):
    if x in ProductAverages:
        return ProductAverages[x]
    else:
        return 0

In [382]:
df['Product Mean Price'] = df['Product_ID'].apply(ProductMean)
df_test['Product Mean Price'] = df_test['Product_ID'].apply(ProductMean)

In [383]:
ProductCount = df.Product_ID.value_counts().to_dict()
ProductCount
ProductCount_test = df_test.Product_ID.value_counts().to_dict()

In [384]:
def ProductCounter(x):
    return ProductCount[x]

def ProductCounter_test(x):
    return ProductCount_test[x]

In [385]:
df['Product Count'] = df['Product_ID'].apply(ProductCounter)
df_test['Product Count'] = df_test['Product_ID'].apply(ProductCounter_test)

In [149]:
def CityConverter(x):
    if x == 'A':
        return 1
    elif x == 'B':
        return 2
    else :
        return 3

In [386]:
df.City_Category = df['City_Category'].apply(CityConverter)
df_test.City_Category = df_test['City_Category'].apply(CityConverter)

In [152]:
from sklearn.preprocessing import LabelEncoder

In [159]:
le = LabelEncoder()

In [160]:
le.fit(df['Product_ID'])

LabelEncoder()

In [162]:
df['Product Encoded'] = le.transform(df['Product_ID'])

In [164]:
le.classes_.shape

(3631,)

In [165]:
df.drop(columns = 'Product_ID',axis=1, inplace = True)

In [387]:
le = LabelEncoder()

le.fit(df_test['Product_ID'])

df_test['Product Encoded'] = le.transform(df_test['Product_ID'])

le.classes_.shape

df_test.drop(columns = 'Product_ID',axis=1, inplace = True)

In [167]:
le.fit(df['User_ID'])

LabelEncoder()

In [168]:
df['User_ID'] = le.transform(df['User_ID'])

In [388]:
le.fit(df_test['User_ID'])

df_test['User_ID'] = le.transform(df_test['User_ID'])

In [389]:
df = pd.get_dummies(df,prefix=['City'],columns=['City_Category'])
df_test = pd.get_dummies(df_test,prefix=['City'],columns=['City_Category'])

In [288]:
def boolInt(x):
    if x == True:
        return 1
    else :
        return 0

In [391]:
df['HighPrice']  = df['Purchase'] > df['Product Mean Price']
df['HighPrice'] = df['HighPrice'].apply(boolInt)


In [335]:

RichDict = dict()
for i in range (0,df['User_ID'].nunique()):
    RichDict.update({i : (df[df.User_ID == i].HighPrice.sum()/df[df.User_ID == i].HighPrice.count())})

In [346]:
def UserRich(x):
    return RichDict[x]

In [392]:
df['HighPay'] = df['User_ID'].apply(UserRich)
df_test['HighPay'] = df_test['User_ID'].apply(UserRich)

In [353]:
df.drop(columns = 'HighPrice', axis = 1, inplace = True)

In [354]:
X = df.drop(columns = 'Purchase', axis = 1)
X.drop(columns = 'Product Encoded',axis =1, inplace = True)
Y = df['Purchase']


<h2>Model Building</h2>

In [355]:
from sklearn.model_selection import train_test_split

In [356]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2, random_state=101)

In [398]:
X_test_actual = df_test
X_test_actual.head()
X_test_actual.drop(columns = 'Product Encoded',axis =1, inplace = True)

In [358]:
from xgboost import XGBRegressor

In [412]:
XGB = XGBRegressor(objective="reg:linear",nrounds=500,max_depth=10,eta=0.1,colsample_bytree=0.5,seed=69,metric="rmse",importance=1)

In [413]:
XGB = XGB.fit(X_train,Y_train)

<h2>Predicting Results</h2>

In [ ]:
Y_pred = XGB.predict(X_test)

In [369]:
from sklearn.metrics import mean_squared_error

In [ ]:
import math
math.sqrt(mean_squared_error(Y_test,Y_pred))

In [414]:
Y_submit = XGB.predict(X_test_actual)

In [401]:
Y_submit

array([16604.793 , 10472.467 ,  6504.1846, ...,  8486.231 , 16596.375 ,
        2146.5186], dtype=float32)

<h2>Creating Submission File</h2>

In [404]:
df_submit = pd.read_csv("test.csv")
submit = pd.DataFrame()

In [405]:
submit['User_ID'] = df_submit['User_ID']
submit['Product_ID'] = df_submit['Product_ID']

In [415]:
submit['Purchase'] = Y_submit

In [416]:
submit.head()

,User_ID,Product_ID,Purchase
0,1000004,P00128942,14350.472656
1,1000009,P00113442,10686.152344
2,1000010,P00288442,7541.604980
3,1000010,P00145342,2108.221191
4,1000011,P00053842,2679.523438


In [417]:
submit.to_csv("Hackathon_Submisson.csv",index=False)

<h2> Dumping Model to Pickle<h2>

In [421]:
import pickle


In [423]:
with open ('hackathon_model_pickle','wb')  as f:
    pickle.dump(XGB,f)